In [1]:
import wfdb
import pandas as pd
import numpy as np
import os
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.dists_kernels.dtw import DtwDtaidistMultiv

In [2]:
data_abs_path = '/Users/victor/physionet.org/files/mimic-iv-ecg/1.0/files'

split_csv = pd.read_csv('data/MIMIC-IV-ECG-Ext-Electrolytes/few_shot_splits/128shots/split1/Calcium50893.csv')
split_csv = split_csv[split_csv['subject_id'] <= 10400000]

classifiers = pd.read_csv('data/MIMIC-IV-ECG-Ext-Electrolytes/mimiciv_ECGv1.1_hospV2.2_Calcium50893.csv')

train = []
test = []
val = []

for index, row in split_csv.iterrows():
    path = data_abs_path + '/p' + f"{row['subject_id']}"[:4] + '/p' + f"{row['subject_id']}"
    samples = os.listdir(path)


    for sample in samples:
        sample_path = os.path.join(path, sample)
        if os.path.isdir(sample_path):
            if int(sample[1:]) not in classifiers['study_id'].values:
                continue
            signal, fields = wfdb.rdsamp(sample_path + '/' + sample[1:])
            if row['split'] == 'train':
                train.append(signal)
            elif row['split'] == 'test':
                test.append(signal)
            elif row['split'] == 'val':
                val.append(signal)

In [24]:
combined_data = [[None for _ in range(12)] for _ in range(5000)]

for row in range(5000):
    for col in range(12):
        values = [trial[row][col] for trial in train]
        combined_data[row][col] = pd.DataFrame(values)

X_train = pd.DataFrame(combined_data)
X_train

,0,1,2,3,4,5,6,7,8,9,10,11
0,0 0 -0.020 1 -0.015 2 0.010 3 -0.045 4...,0 0 0.010 1 -0.080 2 0.000 3 -0.065 4...,0 0 0.025 1 -0.055 2 -0.005 3 -0.010 4...,0 0 0.005 1 0.050 2 -0.005 3 0.055 4...,0 0 0.015 1 -0.065 2 -0.005 3 -0.040 4...,0 0 -0.025 1 0.025 2 0.015 3 -0.020 4...,0 0 0.000 1 0.015 2 0.010 3 0.000 4...,0 0 -0.035 1 0.320 2 0.075 3 0.005 4...,0 0 -0.060 1 0.155 2 -0.010 3 0.010 4...,0 0 -0.100 1 0.135 2 -0.050 3 0.015 4...,0 0 -0.020 1 -0.030 2 -0.090 3 -0.025 4...,0 0 -0.010 1 -0.095 2 -0.030 3 0.045 4...
1,0 0 -0.015 1 -0.025 2 0.015 3 -0.060 4...,0 0 0.000 1 -0.075 2 0.020 3 -0.065 4...,0 0 0.010 1 -0.040 2 0.010 3 0.005 4...,0 0 0.005 1 0.050 2 -0.020 3 0.060 4...,0 0 0.005 1 -0.055 2 0.015 3 -0.030 4...,0 0 -0.015 1 0.015 2 0.010 3 -0.035 4...,0 0 -0.005 1 0.030 2 0.005 3 0.010 4...,0 0 -0.015 1 0.355 2 0.075 3 0.005 4...,0 0 -0.005 1 0.150 2 -0.020 3 0.010 4...,0 0 -0.035 1 0.140 2 -0.050 3 0.010 4...,0 0 -0.015 1 -0.020 2 -0.090 3 -0.020 4...,0 0 0.000 1 -0.095 2 -0.030 3 0.055 4...
2,0 0 0.010 1 -0.035 2 0.010 3 -0.060 4...,0 0 0.010 1 -0.075 2 0.030 3 -0.060 4...,0 0 -0.005 1 -0.030 2 0.025 3 0.010 4...,0 0 -0.010 1 0.055 2 -0.020 3 0.060 4...,0 0 0.000 1 -0.050 2 0.025 3 -0.025 4...,0 0 0.005 1 0.005 2 0.000 3 -0.035 4...,0 0 0.000 1 0.035 2 0.005 3 0.015 4...,0 0 -0.005 1 0.375 2 0.075 3 0.005 4...,0 0 0.02 1 0.15 2 -0.02 3 0.01 4 -0.04,0 0 0.010 1 0.135 2 -0.050 3 0.005 4...,0 0 -0.015 1 -0.010 2 -0.085 3 -0.020 4...,0 0 0.000 1 -0.090 2 -0.030 3 0.055 4...
3,0 0 0.020 1 -0.045 2 0.000 3 -0.055 4...,0 0 0.025 1 -0.075 2 0.030 3 -0.060 4...,0 0 0.000 1 -0.020 2 0.035 3 0.005 4...,0 0 -0.020 1 0.060 2 -0.015 3 0.055 4...,0 0 0.010 1 -0.045 2 0.030 3 -0.030 4...,0 0 0.005 1 -0.005 2 -0.010 3 -0.030 4...,0 0 0.010 1 0.030 2 0.005 3 0.015 4...,0 0 -0.015 1 0.390 2 0.070 3 0.005 4...,0 0 0.030 1 0.155 2 -0.010 3 0.005 4...,0 0 0.010 1 0.130 2 -0.050 3 -0.005 4...,0 0 -0.010 1 -0.020 2 -0.085 3 -0.020 4...,0 0 0.000 1 -0.085 2 -0.040 3 0.040 4...
4,0 0 0.020 1 -0.035 2 0.000 3 -0.035 4...,0 0 0.030 1 -0.070 2 0.010 3 -0.055 4...,0 0 0.005 1 -0.025 2 0.015 3 -0.010 4...,0 0 -0.025 1 0.055 2 -0.005 3 0.045 4...,0 0 0.015 1 -0.045 2 0.010 3 -0.035 4...,0 0 0.005 1 0.000 2 0.000 3 -0.015 4...,0 0 0.01 1 0.03 2 0.00 3 0.01 4 0.00,0 0 -0.020 1 0.395 2 0.055 3 0.015 4...,0 0 0.000 1 0.155 2 -0.010 3 0.000 4...,0 0 -0.030 1 0.115 2 -0.045 3 -0.015 4...,0 0 -0.015 1 -0.025 2 -0.085 3 -0.025 4...,0 0 0.000 1 -0.075 2 -0.040 3 0.040 4...
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0 0 -0.010 1 0.065 2 0.065 3 -0.015 4...,0 0 -0.025 1 0.120 2 0.070 3 0.030 4...,0 0 -0.020 1 0.065 2 0.010 3 0.055 4...,0 0 0.015 1 -0.090 2 -0.065 3 -0.010 4...,0 0 -0.025 1 0.095 2 0.035 3 0.040 4...,0 0 0.005 1 0.005 2 0.035 3 -0.040 4...,0 0 0.000 1 -0.095 2 -0.015 3 -0.035 4...,0 0 -0.095 1 0.050 2 0.055 3 -0.030 4...,0 0 -0.21 1 0.03 2 0.06 3 0.04 4 -0.20,0 0 -0.110 1 0.370 2 0.090 3 0.055 4...,0 0 -0.04 1 0.20 2 0.08 3 0.11 4 0.52,0 0 -0.030 1 0.120 2 0.085 3 0.060 4...
4996,0 0 -0.010 1 0.055 2 0.060 3 -0.005 4...,0 0 -0.025 1 0.115 2 0.070 3 0.035 4...,0 0 -0.020 1 0.070 2 0.015 3 0.050 4...,0 0 0.015 1 -0.080 2 -0.065 3 -0.015 4...,0 0 -0.025 1 0.095 2 0.040 3 0.040 4...,0 0 0.005 1 -0.005 2 0.030 3 -0.030 4...,0 0 0.000 1 -0.085 2 -0.010 3 -0.035 4...,0 0 -0.090 1 0.045 2 0.050 3 -0.035 4...,0 0 -0.21 1 0.03 2 0.06 3 0.04 4 -0.24,0 0 -0.090 1 0.345 2 0.080 3 0.055 4...,0 0 -0.035 1 0.205 2 0.070 3 0.090 4...,0 0 -0.025 1 0.125 2 0.070 3 0.055 4...
4997,0 0 -0.015 1 0.040 2 0.050 3 0.005 4...,0 0 -0.030 1 0.110 2 0.070 3 0.035 4...,0 0 -0.020 1 0.080 2 0.025 3 0.040 4...,0 0 0.02 1 -0.07 2 -0.06 3 -0.02 4 -0.18,0 0 -0.025 1 0.095 2 0.045 3 0.035 4...,0 0 0.000 1 -0.015 2 0.020 3 -0.020 4...,0 0 0.000 1 -0.080 2 -0.010 3 -0.035 4...,0 0 -0.085 1 0.050 2 0.045 3 -0.030 4...,0 0 -0.215 1 0.035 2 0.045 3 0.040 4...,0 0 -0.055 1 0.315 2 0.070 3 0.035 4...,0 0 -0.035 1 0.220 2 0.065 3 0.070 4...,0 0 -0.020 1 0.140 2 0.065 3 0.0

In [14]:
subject_ids = split_csv['subject_id']
filtered_classifiers = classifiers[classifiers['subject_id'].isin(split_csv[split_csv['split'] == 'train']['subject_id'])]
y_train = filtered_classifiers['flag']
y_train

763     abnormal
1000         NaN
1509    abnormal
1510    abnormal
1650         NaN
Name: flag, dtype: object

In [15]:
knn_classifier = KNeighborsTimeSeriesClassifier(distance=DtwDtaidistMultiv())
knn_classifier.fit(X_train, y_train)

TypeError: X must be in an sktime compatible format. Allowed scitypes for classifiers are Panel mtypes, for instance a pandas.DataFrame with MultiIndex and last(-1) level an sktime compatible time index. Allowed compatible mtype format specifications are: ['nested_univ', 'numpy3D', 'numpyflat', 'pd-multiindex', 'pd-wide', 'pd-long', 'df-list', 'gluonts_ListDataset_panel', 'gluonts_PandasDataset_panel', 'polars_panel'] . See the data format tutorial examples/AA_datatypes_and_datasets.ipynb. If you think the data is already in an sktime supported input format, run sktime.datatypes.check_raise(data, mtype) to diagnose the error, where mtype is the string of the type specification you want. Error message for checked mtypes, in format [mtype: message], as follows: [df-list: obj must be list of pd.DataFrame, found <class 'pandas.core.frame.DataFrame'>]  [nested_univ: obj entries must be pd.Series]  [numpy3D: obj must be a numpy.ndarray, found <class 'pandas.core.frame.DataFrame'>]  [pd-multi